# YOLO V3

#### batch normalization

* 用于将分布在较大空间内的数据整合到一个比较小的数据空间内, 减少经过非线性以后的数据饱和. 

* 建议在每一个卷积层和ReLU层之间都加入这个层

![bn](bn.png)

输入层是一个大范围内分布的离散数值, 先计算他们的算术平均值, 再计算方差, 经过正则化处理, $\epsilon$是一个极小量, 用于防止方差为零. 最后再进行一个线性预测, 并加入bias $\beta$. 其中, 需要机器学习的量仅为$\gamma$和$\beta$